In [13]:
import neuroglancer
import cloudvolume
import pandas as pd, os

## README
This example assumes you have precomputed datasets already made and hosted with cloudvolume.
To host the raw data in this example that I already converted to precomputed format, open an ipython terminal and do:
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///jukebox/scratch/zmd/contra_ipsi_projection_studies_20191125/20160822_tp_bl6_crii_1500r_06/647')
vol.viewer(port=1337) # You can choose the port here, but remember it for later
```
The last command will cause the window to hang -- that is the expected behavior. Do not try to do this in this jupyter notebook as it will cause your session to hang and you won't be able to run any of the following steps.

## Set the neuroglancer client you want to use. 
Default is Seung lab's which should work fine for this example

In [14]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

# Load in the raw data into neuroglancer and generate the link

In [15]:
# This volume handle can be used to notify the viewer that the data has changed.
viewer = neuroglancer.Viewer()

# Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume
with viewer.txn() as s:
    s.layers['20160822_tp_bl6_crii_1500r_06'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 100.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )    
#loading multiple brains in the same viewer    
    s.layers['20161205_tp_bl6_lob45_1000r_01'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1339',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 40.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
    s.layers['20170116_tp_bl6_lob6b_lpv_07'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1340',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 15.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
    s.layers['20170207_db_bl6_crii_1300r_02'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1341',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 40.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
    s.layers['20170411_db_bl6_crii_mid_53hr'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1342',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 60.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
    s.layers['20170419_db_bl6_cri_rpv_53hr'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1343',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 40.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
print(viewer) # Click the link that is generated to see your volume displayed in Neuroglancer

http://127.0.0.1:35777/v/e76f7f374e23390f98af98ff0d2c250d3709be97/


## Host the raw-space allen atlas cloudvolume

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///jukebox/scratch/zmd/contra_ipsi_projection_studies_20191125/20160822_tp_bl6_crii_1500r_06/atlas')
vol.viewer(port=1338) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

## Load in the atlas
Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume using the same viewer object that you already made. If you go back to the neuroglancer window after running this it should be there in a new layer

In [20]:
with viewer.txn() as s:
    s.layers['rawatlas_20160822_tp_bl6_crii_1500r_06'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    

Use the look-up table to make a key binding to the Neuroglancer session.

In [24]:
# First you need to create a dictionary mapping the atlas id to the region name. 
# Here's my example

csv_file = '/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts_16bit.xlsx'

df = pd.read_excel(csv_file)

atlas_df = df.copy()

atlas_dict = {}
for ind, row in atlas_df.iterrows():
    atlas_dict[int(row['id'])] = row['name']
    
# Here is the actual code for making the key binding. Copy and paste this
# into your jupyter notebook where you have already made the viewer() object

num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['rawatlas_20160822_tp_bl6_crii_1500r_06']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                    (region_id, region_name))

    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'

Got my-action 1
  Layer selected values: Map({"20160822_tp_bl6_crii_1500r_06": 182, "rawatlas_20160822_tp_bl6_crii_1500r_06": 771})


Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError
Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-23-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


Got my-action 4
  Layer selected values: Map({"20160822_tp_bl6_crii_1500r_06": 888, "rawatlas_20160822_tp_bl6_crii_1500r_06": 350})


Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError
Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-23-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


Got my-action 7
  Layer selected values: Map({"20160822_tp_bl6_crii_1500r_06": 389, "rawatlas_20160822_tp_bl6_crii_1500r_06": 358})


...

Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError
Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-23-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


Got my-action 64
  Layer selected values: Map({"20160822_tp_bl6_crii_1500r_06": 911, "rawatlas_20160822_tp_bl6_crii_1500r_06": 280})


Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError
Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-23-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


Got my-action 67
  Layer selected values: Map({"20160822_tp_bl6_crii_1500r_06": 620, "rawatlas_20160822_tp_bl6_crii_1500r_06": 898})


Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError
Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-23-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


## Load cells (optional) 

Load the cells cloud volume in the same viewer as an additional layer

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/20170115_tp_bl6_lob6b_ml_04/cells')
vol.viewer(port=1339) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

In [29]:
with viewer.txn() as s:
    s.layers['cells'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339',
    )

Got my-action 276
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 3665, "rawatlas": 733, "cells": 0})
Got my-action 277
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 3460, "rawatlas": 1020, "cells": 0})
Got my-action 278
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 5529, "rawatlas": 262, "cells": 0})
Got my-action 279
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 1657, "rawatlas": 6, "cells": 0})
Got my-action 280
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 406, "rawatlas": 301, "cells": 0})
Got my-action 281
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 849, "rawatlas": 733, "cells": 0})
Got my-action 282
  Layer selected values: Map({"20170115_tp_bl6_lob6b_ml_04": 333, "rawatlas": 718, "cells": 0})


## Take screenshots (optional)

Adjust the viewer and take a screenshot. Good for videos!

In [18]:
svdst = '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas'
#make sure these directories exist
if not os.path.exists(os.path.dirname(svdst)): os.mkdir(os.path.dirname(svdst)) #brain directory
if not os.path.exists(svdst): os.mkdir(svdst) #structure directory
for i in range(180, 530):
    with viewer.config_state.txn() as s:
        s.show_ui_controls = False
        s.show_panel_borders = False
    with viewer.txn() as s:
        s.voxel_coordinates = [3081, 1314,i]
#    with viewer.config_state.txn() as s:
#        s.viewer_size = [1000,1000]
    ss = neuroglancer.ScreenshotSaver(viewer, svdst)
    ss.capture(index=i)   

Alternatively, can also take a single view screenshot of the current view.

In [10]:
svdst = '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas'
if not os.path.exists(svdst): os.mkdir(svdst)
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
    s.show_panel_borders = False
    s.viewer_size = [1000, 1000]
ss = neuroglancer.ScreenshotSaver(viewer, svdst)
ss.capture(index=0) 

(0,
 '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas/0000000.png')

Return viewer state to normal:

In [23]:
with viewer.config_state.txn() as s:
    s.show_ui_controls = True
    s.show_panel_borders = True
    s.viewer_size = None

Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [22]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20160822_tp_bl6_crii_1500r_06', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas_20160822_tp_bl6_crii_1500r_06'])])

Traceback (most recent call last):
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/viewer_config_state.py", line 94, in invoke
    handler(state)
  File "<ipython-input-21-0c7d8a946b10>", line 22, in my_action3
    region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
  File "/home/wanglab/anaconda2/envs/lspy35/lib/python3.7/site-packages/neuroglancer/json_wrappers.py", line 209, in __getitem__
    raise KeyError
KeyError


Remove the overlay layer.

In [17]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20160822_tp_bl6_crii_1500r_06']),
         neuroglancer.LayerGroupViewer(layers=['20170207_db_bl6_crii_1300r_02']),
        #neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01']),
        neuroglancer.LayerGroupViewer(layers=['20170411_db_bl6_crii_mid_53hr']),
        neuroglancer.LayerGroupViewer(layers=['20170419_db_bl6_cri_rpv_53hr']),
        neuroglancer.LayerGroupViewer(layers=['20170116_tp_bl6_lob6b_lpv_07']),])